In [55]:
import MySQLdb
import sshtunnel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from ast import literal_eval
import matplotlib.pyplot as plt
import glob
from pathlib import Path

In [59]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password='midgeq-xikFah-gukpu7',
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd='david2202087',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        query = "SELECT * FROM pilot_2 where submit_time>='2024-01-01';"
        df = pd.read_sql(query, connection)
        print(f"Data read finished, length {len(df)}")
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

df.head()
df["pilot_2_answers"] = df["pilot_2_answers"].map(literal_eval)

Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished, length 10


In [60]:
ids_attention_failed = df.loc[df['attention_passed'] == 0]["participantId"].tolist()
print(f"Participants who failed attention check: {ids_attention_failed}")
df = df.loc[:, ["participantId", "pilot_2_answers", "ideology_label"]].set_index("participantId")

def IdeologyToNumber(ideology):
    if ideology == "Moderate":
        return 2
    elif ideology == "Somewhat Conservative":
        return 3
    elif ideology == "Conservative":
        return 4
    elif ideology == "Somewhat Liberal": 
        return 1
    elif ideology == "Liberal":
        return 0

connect_data_csv = glob.glob('data/connect_raw_data/pilot_2_2*.csv')[0]
df_connect = pd.read_csv(connect_data_csv).set_index("ParticipantId")
df["connect_ideology"] = df_connect["Political Ideology"].map(IdeologyToNumber)
print("rmse of ideology:", np.sqrt(np.mean((df["ideology_label"] - df["connect_ideology"])**2)))
df["ideology"] = df["ideology_label"]
df.drop(["connect_ideology", "ideology_label"], axis=1, inplace=True)
df.head()

Participants who failed attention check: []
rmse of ideology: 0.8366600265340756


,pilot_2_answers,ideology
participantId,,
3B8488407EB0403686FDCAFEAABB5AFA,"[{'answer': -1.1, 'idx_of_question': 5}, {'ans...",2
6F825AF773F0493A95DA1CC2C5B93009,"[{'answer': 1.1, 'idx_of_question': 7}, {'answ...",2
7378E75D71394AE8B91F1EBD2E13861F,"[{'answer': 1.2, 'idx_of_question': 7}, {'answ...",3
83A21B737F6E497DB9804B866B6536D6,"[{'answer': 1.8, 'idx_of_question': 7}, {'answ...",3
B0E250FD20A94B4484FF07F9AD38A338,"[{'answer': 2.9, 'idx_of_question': 5}, {'answ...",3


## 数据分析
### 0. 需要分析什么
1. 每个问题的答案分布
   - 画频方图（分布）
   - 均值
   - 方差
   - 和意识形态的相关系数
2. 每个问题在主题中的次序对它的影响
   - 均值
   - 方差

### 1. 每个问题的答案分布

In [61]:
def IndexToType(index):
    if index >= 0 and index <= 3:
        return "prediction"
    elif index >= 4 and index <= 7:
        return "issue"
    elif index == 8 or index == 9:
        return "fact"

answers = {"index": [], "seq": [], "answer": [], "ideology": []}
for id, row in df.iterrows():
    pilot_2_answers = row["pilot_2_answers"]
    ideology = row["ideology"]
    tmp_seq = {"prediction": 0, "issue": 0, "fact": 0}
    for each_answer in pilot_2_answers:
        answers["index"].append(each_answer["idx_of_question"])
        answers["answer"].append(each_answer["answer"])
        answers["ideology"].append(ideology)
        type = IndexToType(each_answer["idx_of_question"])
        answers["seq"].append(tmp_seq[type])
        tmp_seq[type] += 1
    
df_answers = pd.DataFrame(answers)
df_answers.head()

,index,seq,answer,ideology
0,5,0,-1.1,2
1,2,0,1.6,2
2,6,1,2.0,2
3,3,1,-0.7,2
4,4,2,-0.2,2


In [ ]:
# 均值
df_answers.groupby("index")["answer"].mean()

index
0   -0.080000
1    0.620000
2    1.070000
3    0.370000
4    0.240000
5   -0.290000
6    0.020000
7    0.060000
8   -0.166667
9    0.350000
Name: answer, dtype: float64

In [ ]:
# 方差
df_answers.groupby("index")["answer"].std()

index
0    1.756765
1    1.764338
2    1.541320
3    1.953942
4    2.217707
5    1.895287
6    2.007099
7    1.362351
8    1.477151
9    2.104757
Name: answer, dtype: float64

In [ ]:
# 和意识形态的相关系数
df_answers.groupby("index").apply(lambda x: pearsonr(x["answer"], x["ideology"])[0])

index
0   -0.024001
1    0.519233
2    0.547733
3    0.647854
4    0.795924
5    0.542514
6    0.790638
7    0.673849
8    0.278952
9    0.594148
dtype: float64

In [ ]:
# 画图
